# Getting Votes From ProductHunt API
The following script will let you gather all usernames from people who upvoted a certain product on ProductHunt. The only thing you'll need is a link to the product you're interested in.

Let's get started!

First, we'll need to set-up the environment.

In [ ]:
import requests
import json
import time

# only if using ALTERNATIVE AUTH
#import yaml

print('Module import - DONE')

Next, you'll have to enter the URL of the product you're interested in. Note that a product's link is preceded by `/posts/` and the link has to be in quotation marks.

This cell will output the product's name, just to check if everything works. If you see output that does not resemble the product's name, make sure that the link you entered follows the pattern `https://www.producthunt.com/posts/product-name`

In [ ]:
# your product's url in quotation marks
prod_url = "https://www.producthunt.com/posts/pivot-bed"

prod_name = prod_url.split('/')
prod_name = prod_name[-1]
print('Product selected:' + prod_name)

To access the API, you'll have to provide a password. To get a password, you need to register your application, in this case - the script you're using at the moment. This can be done if you log in your ProductHunt account and go [here](https://api.producthunt.com/v2/oauth/applications/new). 

Name your application, and provide a redirect URI. Since we won't be actually using it, you can write something like `https://localhost:3000/ `

After this you'll need to click `create token` under Developer token section. Copy the token here, replacing the `'YOUR-DEVELOPER-TOKEN'` placeholder. Your token will be a string of random letters and numbers. Note, that the token needs to be in quotation marks. 

In [ ]:
# ALTERNATIVE AUTH:
# with open("config.yml", 'r') as file:
#    yml = yaml.safe_load(file)
#
# get access token with yml['access_token']

# register developer token here: https://api.producthunt.com/v2/oauth/applications/new
access_token = 'YOUR-DEVELOPER-TOKEN'
print('Adding access token - DONE')

Next, the script connects to ProductHunt's API, loads a function that will be needed later, and initializes the variables.

In [ ]:
url = "https://api.producthunt.com/v2/api/graphql"

headers = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer '+ access_token,
    'Host': 'api.producthunt.com',
    }

# flattens the nested json dictionary, courtesy of a.ziai
def flatten_json(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

condition = True
cursor = ''
user_list = []

print('Initialization - DONE')

The next cell is what gets the usernames. Note that depending on the vote count, it may take some time to run. For example, around 350 votes and their corresponding usernames take around a minute to download. So, be patient.

In [ ]:
while condition is True:
    
    query = """
    {
    post(slug:\"""" + str(prod_name) + """\") {
        votes(first:20, after:\"""" + str(cursor) + """\") {
          pageInfo{
            hasNextPage
            endCursor
          }
            edges {
                node {
                  user {
                    username
                      }
                    }
                  }
            }
        }
    }
    """

    r = requests.post(url, json={'query': query}, headers = headers)
    
    if r.status_code == 200:
        json_data = json.loads(r.text)
        temp_json = flatten_json(json_data)
        temp_list = list(temp_json.values())
        
        # checks if there is a next page
        condition = temp_list[0]
        
        if condition is None:
            for i in range(2, len(temp_list)):
                user_list.append(temp_list[i])
            break
            
        if condition is False:
            for i in range(2, len(temp_list)):
                user_list.append(temp_list[i])
            break

        # last entry in the page
        cursor = temp_list[1]

        # adds users to list
        for i in range(2, len(temp_list)):
            user_list.append(temp_list[i])
    
    # checks if too many requests have been sent
    if r.status_code == 429:
        g = r.headers
        if int(g['X-Rate-Limit-Remaining']) <= 0:
            # waits for n seconds until rate limit resets
            print('You have exceeded the API rate limit, try again after ' + g['X-Rate-Limit-Reset'] + ' seconds.')
            time.sleep(int(g['X-Rate-Limit-Reset']))
            print('Wait over. Restart the kernel and try again.')
            
print('Accessing API - DONE')

If you receive the message that you've exceeded the API rate limit, the program will wait however many seconds are needed until the limit resets. After waiting, you'll need to restart the kernel or refresh the page to resume getting usernames. However, you can still go to the `user_names` section and print out all of the usernames that have been collected before the rate-limit was reached.

ProductHunt's rate limit restrictions are quite vague, and according to its documentation, can also be modified according to API's traffic. On an isolated occurrence, I've downloaded over 1000 usernames in under 15 minutes with this script, so there shouldn't be many rate-limit concerning errors with regular usage, as this API request is fairly simple. 

The next cell prints out the amount of usernames you got.

In [ ]:
print('Downloaded ' + len(user_list) + 'usernames')

Finally, the last cell prints out the username list of all users that upvoted a product on ProductHunt. 

In [ ]:
user_list